In [2]:
import json
from azure.ai.formrecognizer import FormRecognizerClient
from azure.ai.formrecognizer import FormTrainingClient
from azure.core.credentials import AzureKeyCredential
import os, requests
from dotenv import load_dotenv


In [3]:
body = {
    "values": [
        {   "recordId": "record1",
            "data": { 
                "formUrl": "https://raw.githubusercontent.com/Azure-Samples/cognitive-services-REST-api-samples/master/curl/form-recognizer/rest-api/layout.png",
                "formSasToken": ""}
        }
    ]
}
json_data = json.dumps(body)
values = json.loads(json_data)['values'] 

## Adding FR metadata 

In [8]:
load_dotenv('..')
cog_endpoint = os.getenv('COGNITIVE_SERVICE_FORM_RECOGNIZER_ENDPOINT')
cog_key = os.getenv('COGNITIVE_SERVICE_FORM_RECOGNIZER_KEY')
model= "prebuilt-layout"
form_recognizer_url = f"{cog_endpoint}formrecognizer/documentModels/{model}:analyze?api-version=2022-08-31"


## Here comes the transform_value part



In [12]:
# extracting one value metadata
value=values[0]
data = value["data"]
form_url = data["formUrl"]  + data["formSasToken"]   
recordId = value['recordId']


# Making a placeholder for the service bus message
output=[]
output = {
            "key": "",
            "file_path": form_url,
            "model": model,
            "Operation-location":""
        } 

# Request to Azure Form Recognizer Model
headers = {"Content-Type": "application/json", "Ocp-Apim-Subscription-Key": cog_key}
body = {'urlSource': output['file_path']}
r = requests.post(form_recognizer_url, headers=headers, json=body)

print(r.status_code)
# Adding last bit to the service bus message
if r.status_code<300:
    output['Operation-location'] = r.headers['Operation-Location']
else: output['Operation-location']=f"code: {r.status_code}"
json.dumps(output)


202


'{"key": "", "file_path": "https://raw.githubusercontent.com/Azure-Samples/cognitive-services-REST-api-samples/master/curl/form-recognizer/rest-api/layout.png", "model": "prebuilt-layout", "Operation-location": "https://formr-pdf.cognitiveservices.azure.com/formrecognizer/documentModels/prebuilt-layout/analyzeResults/7dc41909-61db-4e9d-985e-3243569f3f03?api-version=2022-08-31"}'